In [ ]:
https://us.download.nvidia.com/XFree86/Linux-x86_64/550.78/NVIDIA-Linux-x86_64-550.78.run# For Youtube -----
# mount Google Drive (you can do it by clicking on it's icon!)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
links = """
https://fedorapeople.org/groups/virt/virtio-win/direct-downloads/stable-virtio/virtio-win.iso
"""


f720 = True
#f720 = False

# zip = False
zip = True

public = True
# public = False

# method = 'ytdlp'
# method = 'pytube'
# method = 'youtube-dl-music'
# method = 'youtube-dl'
method = 'wget'
# method = 'torrent-magnet'

In [2]:
import datetime,sys

links = links.splitlines()
# remove empty elemets

links = [element for element in links if element]

basename = "download"
suffix = datetime.datetime.now().strftime("%y%m%d_%H%M%S")
tmpDir = basename+'_'+suffix
# create temp folder in Colab for save files during download proccess.
!mkdir -p "$tmpDir"
zipPass = "hasanhasan"

def ytdlpGet(links, switches = ''):
  # !python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
  #!python3 -m pip install https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
  !python3 -m pip install yt-dlp

  for link in links:
    !yt-dlp $switches "$link"

def youtubeDlGet(links, customFlag = '', ytdlp = False):
  !pip install youtube-dl

  format = " -f 'bestvideo[height<=720]+bestaudio/best[height<=720]'" if f720 else ""
  if customFlag == '':
    customFlag = format + " --write-auto-sub --sub-lang en,fa --embed-subs --write-sub"

  switches = customFlag + " -o '" + tmpDir + "/%(playlist)s/%(playlist_index)s-%(title)s-%(id)s.%(ext)s'"

  if ytdlp:
    ytdlpGet(links, switches)

  else:
    errorLinks = [];
    for link in links:
        try: #NOT WORKING!
            !youtube-dl $switches "$link"
          # !youtube-dl $customFlag --external-downloader aria2c \
          # --external-downloader-args '-x 8 -s 8 -k 1M' -o "$tmpDir/%(title)s-%(id)s.%(ext)s" "$link"
        except:
          errorLinks.append(link)

    if(len(errorLinks) > 0):
      ytdlpGet(errorLinks)

if method == 'youtube-dl':
  youtubeDlGet(links)

elif method == 'ytdlp':
  youtubeDlGet(links, '', True)

elif method == 'youtube-dl-music':
  youtubeDlGet(links, "-x --audio-format mp3", True)

elif method == 'pytube':
  !pip install pytube
  # !python -m pip install git+https://github.com/ifahadone/pytube
  from pytube import YouTube
  errorLinks = [];

  for link in links:
    try:
      video = YouTube(link)
      download = video.streams.filter(res="720p")
      filename = download.first().download(output_path=tmpDir)
      print(filename)
    except:
      errorLinks.append(link)

  if(len(errorLinks) > 0):
    youtubeDlGet(errorLinks)

elif method == 'wget':
  for link in links:
    !wget -c -P "$tmpDir" "$link"

elif method == 'torrent-magnet':
  !python -m pip install --upgrade pip setup wheel
  !python -m pip install lbry-libtorrent
  !apt install python3-libtorrent
  import libtorrent as lt
  ses = lt.session()
  ses.listen_on(6881,6891)
  downloads = []
  params = {"save_path": tmpDir }
  for link in links:
    downloads.append(lt.add_magnet_uri(ses, link , params))

  import time
  from IPython.display import display
  import ipywidgets as widgets

  state_str = [
      "queued",
      "checking",
      "downloading metadata",
      "downloading",
      "finished",
      "seeding",
      "allocating",
      "checking fastresume",
  ]

  layout = widgets.Layout(width="auto")
  style = {"description_width": "initial"}
  download_bars = [
      widgets.FloatSlider(
          step=0.01, disabled=True, layout=layout, style=style
      )
      for _ in downloads
  ]
  display(*download_bars)

  while downloads:
      next_shift = 0
      for index, download in enumerate(downloads[:]):
          bar = download_bars[index + next_shift]
          if not download.is_seed():
              s = download.status()

              bar.description = " ".join(
                  [
                      download.name(),
                      str(s.download_rate / 1000),
                      "kB/s",
                      state_str[s.state],
                  ]
              )
              bar.value = s.progress * 100
          else:
              next_shift -= 1
              ses.remove_torrent(download)
              downloads.remove(download)
              bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
              download_bars.remove(bar)
              print(download.name(), "complete")
      time.sleep(1)

else:
  sys.exit('please select one correct method!')

# create folder in GDrive
folder = "/content/drive/MyDrive/youtube/"

if public:
  folder+="public/"

!mkdir -p "$folder"

if (zip):
  !zip -r -0 -P "$zipPass" "$tmpDir".zip "$tmpDir"/
  !mv "$tmpDir".zip "$folder"
else:
  !mv "$tmpDir"/* "$folder"

!echo "---------------- done ---------------------"

--2024-05-17 13:04:10--  https://fedorapeople.org/groups/virt/virtio-win/direct-downloads/stable-virtio/virtio-win.iso
Resolving fedorapeople.org (fedorapeople.org)... 152.19.134.199, 2600:2701:4000:5211:dead:beef:a7:9474
Connecting to fedorapeople.org (fedorapeople.org)|152.19.134.199|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://fedorapeople.org/groups/virt/virtio-win/direct-downloads/archive-virtio/virtio-win-0.1.240-1/virtio-win.iso [following]
--2024-05-17 13:04:10--  https://fedorapeople.org/groups/virt/virtio-win/direct-downloads/archive-virtio/virtio-win-0.1.240-1/virtio-win.iso
Reusing existing connection to fedorapeople.org:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://fedorapeople.org/groups/virt/virtio-win/direct-downloads/archive-virtio/virtio-win-0.1.240-1/virtio-win-0.1.240.iso [following]
--2024-05-17 13:04:10--  https://fedorapeople.org/groups/virt/virtio-win/direct-downloads/ar

In [ ]:
from pytube import YouTube
YouTube('https://youtu.be/2lAe1cqCOXo').streams.first().download()
yt = YouTube('http://youtube.com/watch?v=2lAe1cqCOXo')
yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download()

In [ ]:
# Sepearate Download

import datetime,sys
basename = "videos"
suffix = datetime.datetime.now().strftime("%y%m%d_%H%M%S")
tmpDir = basename+'_'+suffix
# create temp folder in Colab for save files during download proccess.
!mkdir -p "$tmpDir"

# create folder in GDrive
folder = "/content/drive/MyDrive/youtube/"+suffix
!mkdir -p "$folder"

In [ ]:
!python -m pip install --upgrade pip setup wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

In [ ]:
import libtorrent as lt
ses = lt.session()
ses.listen_on(6881,6891)
downloads = []

In [ ]:
## Add torrent from file
from google.colab import files

source = files.upload()
params = {
	"save_path": "/content/drive/My Drive/Torrent",
	"ti": lt.torrent_info(list(source.keys())[0])
}

downloads.append(ses.add_torrent(params))

In [ ]:
params = {"save_path": "/content/drive/My Drive/Torrent"}
for link in links:
  downloads.append(lt.add_magnet_uri(ses, link , params))


In [ ]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)

In [ ]:
from pytube import YouTube
YouTube('https://youtu.be/2lAe1cqCOXo').streams.first().download()
yt = YouTube('http://youtube.com/watch?v=2lAe1cqCOXo')
yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download()

In [ ]:
video = YouTube("https://www.youtube.com/watch?v=AXzesol2sOA")
hd_videos = video.streams.filter(res='1080p', type='video', subtype='mp4')
v2 = video.streams.filter(res='720p', type='video', subtype='mp4')
v3 = video.streams.filter(res="720p")
try:
  v2.first().download()
except:
  print('hi')

hi
